In [6]:
%pip install peft transformers accelerate bitsandbytes

In [7]:
%pip install datasets

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

tokenizer = AutoTokenizer.from_pretrained("klyang/MentaLLaMA-chat-7B", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "klyang/MentaLLaMA-chat-7B",
    quantization_config=bnb_config,
    device_map="auto"
)
model.gradient_checkpointing_enable()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
!git clone https://github.com/JOJOZYH/MentalLLaMA_distill.git

Cloning into 'MentalLLaMA_distill'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 480 (delta 73), reused 86 (delta 34), pack-reused 354 (from 1)
Receiving objects: 100% (480/480), 27.15 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (248/248), done.


In [19]:
#%cd MentalLLaMA_distill
#!git pull


/content/MentalLLaMA_distill
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 5), reused 13 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (13/13), 716.11 KiB | 8.52 MiB/s, done.
From https://github.com/JOJOZYH/MentalLLaMA_distill
   874a9c6..cbad357  main       -> origin/main
Updating 874a9c6..cbad357
Fast-forward
 .DS_Store                                          | Bin 6148 -> 6148 bytes
 V3_generate_json_data/failed_generate_dr_train.csv |   6 +
 data/.DS_Store                                     | Bin 6148 -> 6148 bytes
 data/data_generated/train/.DS_Store                | Bin 0 -> 6148 bytes
 data/data_generated/train/DR/.DS_Store             | Bin 0 -> 6148 bytes
 data/data_generated/train/DR/generate_dr_train.csv | 431 +++++++++
 data/data_generated/train/DR/generate_dr_val.csv   | 999 +++++++++++++++++++++
 .../train/DR/generated_dataset_dr_train.csv        |  

In [10]:
import pandas as pd
from datasets import Dataset

# Load CSV using pandas
DR_train = pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/DR/generate_dr_train.csv")
DR_val =pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/DR/generate_dr_val.csv")
lrf_train=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/Irf/generated_dataset_Irf_train.csv")
lrf_val=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/SAD/generated_dataset_sad_val.csv")
SAD_train=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/SAD/generated_dataset_sad_train.csv")
SAD_val=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/SAD/generated_dataset_sad_val.csv")
dreaddit_train=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/dreaddit/generated_dataset_dreaddit_train.csv")
dreaddit_val=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/dreaddit/generated_dataset_dreaddit_val.csv")
multiwd_train=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/multiwd/generated_dataset_multiwd_train.csv")
multiwd_val=pd.read_csv("/content/MentalLLaMA_distill/data/data_generated/train/multiwd/generated_dataset_multiwd_val.csv")

train_df = pd.concat([DR_train, lrf_train, SAD_train, dreaddit_train, multiwd_train], ignore_index=True)
eval_df = pd.concat([DR_val, lrf_val, SAD_val, dreaddit_val, multiwd_val], ignore_index=True)


In [11]:
import re

def parse_query(query):
    post_match = re.search(r'Consider this post:\s*"(.*?)"', query, re.DOTALL)
    post = post_match.group(1).strip() if post_match else None

    question_match = re.search(r'Question:\s*(.*)', query)
    question = question_match.group(1).strip() if question_match else None

    return pd.Series([post, question])

def parse_answer(answer):
    reasoning_parts = answer.split("Reasoning:")
    if len(reasoning_parts) >= 2:
        label_text = reasoning_parts[0].strip()
        reasoning = reasoning_parts[-1].strip()
    else:
        label_text = answer.strip()
        reasoning = None

    return pd.Series([label_text, reasoning])


train_df[['post', 'question']] = train_df['query'].apply(parse_query)
train_df[['label', 'reason']] = train_df['full_answer'].apply(parse_answer)

eval_df[['post', 'question']] = eval_df['query'].apply(parse_query)
eval_df[['label', 'reason']] = eval_df['full_answer'].apply(parse_answer)


In [12]:
train_df

,query,full_answer,post,question,label,reason
0,"Consider this post: ""I'm finally dead inside a...","Yes, the poster suffers from depression. Reaso...",I'm finally dead inside and I don't know how t...,Does the poster suffer from depression?,"Yes, the poster suffers from depression.",The poster expresses a profound sense of emoti...
1,"Consider this post: ""what's it like coming off...","Yes, the poster suffers from depression. Reaso...",what's it like coming off anti-depressants? I ...,Does the poster suffer from depression?,"Yes, the poster suffers from depression.",The poster is inquiring about the experience o...
2,"Consider this post: ""I really need help making...","Yes, the poster suffers from depression. Reaso...","I really need help making friends I'm 25, livi...",Does the poster suffer from depression?,"Yes, the poster suffers from depression.",The poster describes a lack of social connecti...
3,"Consider this post: ""So dramatic 😪 My best fri...","No, the poster does not suffer from depression...",So dramatic 😪 My best friend and I were in the...,Does the poster suffer from depression?,"No, the poster does not suffer from depression.",The post focuses on the dynamics of a changing...
4,"Consider this post: ""My dad's mad after game n...","No, the poster does not suffer from depression...",My dad's mad after game night. My family and I...,Does the poster suffer from depression?,"No, the poster does not suffer from depression.",The post describes a family conflict during a ...
...,...,...,...,...,...,...
17948,"Consider this post: ""I've struggled with depre...","Yes, this wellness dimension exists in the pos...",I've struggled with depression for a few years...,Does the emotional wellness dimension exist in...,"Yes, this wellness dimension exists in the post.","Yes, this wellness dimension exists in the pos..."
17949,"Consider this post: ""Can't shake the thoughts....","Yes, this wellness dimension exists in the pos...",Can't shake the thoughts. Sometimes I just go ...,Does the emotional wellness dimension exist in...,"Yes, this wellness dimension exists in the post.","Yes, this wellness dimension exists in the pos..."
17950,"Consider this post: ""I'm stuck in this rut and...","No, this wellness dimension does not exist in ...",I'm stuck in this rut and I can't stop crying ...,Does the spiritual wellness dimension exist in...,"No, this wellness dimension does not exist in ...","No, this wellness dimension does not exist in ..."
17951,"Consider this post: ""I‚Äôm just done at this p...","Yes, this wellness dimension exists in the pos...",I‚Äôm just done at this point. I have to work ...,Does the social wellness dimension exist in th...,"Yes, this wellness dimension exists in the post.","Yes, this wellness dimension exists in the pos..."


In [13]:
#shuffle
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
eval_df = eval_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
train_df.drop(columns=["query","full_answer"], inplace=True)
eval_df.drop(columns=["query","full_answer"], inplace=True)
train_df = train_df.rename(columns={"query": "post"})
eval_df = eval_df.rename(columns={"query": "post"})


In [14]:
train_df

,query,full_answer,post,question,label,reason
0,"Consider this post: ""Today I finally have to g...",This post shows other stress causes. Reasoning...,"Today I finally have to go back and work, and ...",What is the stress cause of this post?,This post shows other stress causes.,The post explicitly mentions the individual's ...
1,"Consider this post: ""at first I was mad, wonde...","Yes, the post shows this risk. Reasoning: 1. T...","at first I was mad, wondering how could he thi...",Does the post show risk of thwarted belongingn...,"Yes, the post shows this risk.",1. The individual reflects on the suicide of a...
2,"Consider this post: ""My boyfriend and I have b...","Yes, the poster suffers from stress. Reasoning...",My boyfriend and I have been together for 2 ye...,Does the poster suffer from stress?,"Yes, the poster suffers from stress.",The post reveals feelings of insecurity and di...
3,"Consider this post: ""Its hard to find doctors ...","No, the poster does not suffer from stress. Re...",Its hard to find doctors that take cash within...,Does the poster suffer from stress?,"No, the poster does not suffer from stress.",While the post mentions challenges like findin...
4,"Consider this post: ""I question my purpose dai...","No, the post does not show this risk. Reasonin...",I question my purpose daily. Will I ever find the,Does the post show risk of perceived burdensom...,"No, the post does not show this risk.",1. The post does not mention any feelings of b...
...,...,...,...,...,...,...
17948,"Consider this post: ""I've been thinking about ...","Yes, the poster suffers from stress. Reasoning...",I've been thinking about this for weeks and ha...,Does the poster suffer from stress?,"Yes, the poster suffers from stress.","Yes, the poster suffers from stress. The post ..."
17949,"Consider this post: ""I just told them I had to...","Yes, the poster suffers from stress. Reasoning...",I just told them I had too much anxiety and br...,Does the poster suffer from stress?,"Yes, the poster suffers from stress.",The post contains explicit indicators of sever...
17950,"Consider this post: ""i'm 6 months pregnant wor...",This post shows the stress cause related to wo...,i'm 6 months pregnant working a very tiring job.,What is the stress cause of this post?,This post shows the stress cause related to work.,The post 'i'm 6 months pregnant working a very...
17951,"Consider this post: ""i was in a relationship w...","Yes, the post shows this risk. Reasoning: 1. T...",i was in a relationship where my gf had severe...,Does the post show risk of thwarted belongingn...,"Yes, the post shows this risk.",1. The individual expresses feelings of confus...


In [15]:
def tokenize_func(examples):
    prompts = [
        f"Post: {post}\nQuestion: {question}\nAnswer: {label}\nReasoning: {reason}"
        for post, question, label, reason in zip(
            examples["post"], examples["question"], examples["label"], examples["reason"]
        )
    ]

    # Tokenize the full prompt
    tokenized = tokenizer(
        prompts,
        padding="max_length",
        truncation=True,
        max_length=512
    )

    # Prepare labels (ignore padding tokens)
    tokenized["labels"] = [
        [token if token != tokenizer.pad_token_id else -100 for token in ids]
        for ids in tokenized["input_ids"]
    ]

    return tokenized


In [16]:
train_df = Dataset.from_pandas(train_df)
eval_df = Dataset.from_pandas(eval_df)
train_df = train_df.map(tokenize_func, batched=True)
eval_df = eval_df.map(tokenize_func, batched=True)

Map:   0%|          | 0/17953 [00:00<?, ? examples/s]

Map:   0%|          | 0/4020 [00:00<?, ? examples/s]

In [17]:
train_dataset = train_df.remove_columns(["post", "question", "label", "reason"])
eval_df_dataset = eval_df.remove_columns(["post", "question", "label", "reason"])

In [18]:
import transformers
print(transformers.__version__) # 如果你的版本小于4.4，请跑以下代码并restart runtime
#!pip install -U transformers #update transformer version

4.51.3


In [20]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [22]:
!pip install wandb
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zhzizhen (zhzizhen-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./lora_mental_llama",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    gradient_accumulation_steps=4,
    optim="adamw_bnb_8bit",
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_df_dataset
)
model.gradient_checkpointing_disable()
model.train()
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebook/fine_tuned_lora_mental_llama/added_tokens.json')